# Example 1: PDF 테이블 정보에 대한 Recursive Retrieval 전략
- 다수의 CSV 테이블 대상으로 검색 chunk와 답변 생성 chunk 분리해보기

In [ ]:
%pip install llama-index-embeddings-openai llama-index-llms-openai camelot-py llama-index

In [ ]:
!apt-get install ghostscript
!pip install ghostscript

In [ ]:
import camelot

# https://en.wikipedia.org/wiki/The_World%27s_Billionaires
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import PandasQueryEngine
from llama_index.core.schema import IndexNode
from llama_index.llms.openai import OpenAI

from llama_index.readers.file import PyMuPDFReader
from typing import List

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = '<YOUR_OPENAI_KEY>'

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# 추후 사용할 llm, 임베딩 모델 클래스 정의
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:
# 파싱할 파일 경로 설정
file_path = "billionaires_page.pdf"

In [ ]:
# PDF파서 정의
reader = PyMuPDFReader()

In [ ]:
# 업로드된 경로에서 로딩스테이지 진행한 후 다큐먼트 단위로 저장
docs = reader.load(file_path)

In [ ]:
# 도큐먼트 정보 확인
docs

In [ ]:
from llama_index.core import Settings
#노드변환 및 파싱
doc_nodes = Settings.node_parser.get_nodes_from_documents(docs)

In [ ]:
vector_index0 = VectorStoreIndex(doc_nodes)
vector_query_engine0 = vector_index0.as_query_engine()

In [ ]:
response = vector_query_engine0.query(
    "How many billionaires were there in 2009?"
)

In [ ]:
print(response.source_nodes[0].node.get_content())

In [ ]:
print(str(response))

In [ ]:
response = vector_query_engine0.query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

In [ ]:
print(response.source_nodes[0].node.get_content())

- 기본적인 PDF파싱모듈로는 테이블 등 Text-Only 가 아닌 문서에 대한 정보 해석력이 떨어지는 것을 확인
- Table정보를 따로 추출하여 답하는 방식은 어떨지?

In [ ]:
# pdf의 테이블파싱하기
def get_tables(path: str, pages: List[int]):
    table_dfs = []
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        for table in table_list:
            table_df = table.df
            table_df = (
                table_df.rename(columns=table_df.iloc[0])
                .drop(table_df.index[0])
                .reset_index(drop=True)
            )
            table_dfs.append(table_df)
    return table_dfs

In [ ]:
table_dfs = get_tables(file_path, pages=[3,4,24])

In [ ]:
#파싱된 테이블 개수확인
len(table_dfs)

In [ ]:
#파싱 결과 확인
table_dfs[0]

In [ ]:
#파싱 결과 확인
table_dfs[1]

In [ ]:
#파싱 결과 확인
table_dfs[-1]

이제 테이블을 다 파싱해왔는데,
이것들을 기반으로 질문에 바로 답할수 있도록 만들려면 만들수도 있겠지만,
테이블이 지금과 다르게 수천 수만개일때, 모든 유저 쿼리에 대해 수만개의 테이블을 매번 조회하는 것은 실용성 없는 Naive한 접근방식(자원은 무한하지 않다).

그렇기 때문에,
1. 사용자의 질문과 관련된 테이블을 먼저 찾고
2. 찾은 테이블을 기준으로 사용자의 질문에 답할 수 있는 정보를 발췌하여 답해보자.

일단은 각 테이블별로 답해주는 담당 라마인덱스 쿼리엔진을 만들어주자.

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")

df_query_engines = [
    PandasQueryEngine(table_df, llm=llm) for table_df in table_dfs
]

In [ ]:
# 상응하는 테이블 지정해서 답변 요구
response = df_query_engines[0].query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

In [ ]:
# 상응하는 테이블 지정해서 답변 요구
response = df_query_engines[1].query(
    "What's the net worth of the second richest billionaire in 2022?"
)
print(str(response))

In [ ]:
response = df_query_engines[4].query(
    "How many billionaires were there in 2009?"
)
print(str(response))

질문별로 담당하는 쿼리엔진을 부여하는 것으로 heuristic하게 서칭 스페이스를 줄이고 시작할 수 있는 것 확인

In [ ]:
# 쿼리엔진 요약문 생성
summaries = [
    (
        "This node provides information about the world's richest billionaires"
        " in 2023"
    ),
    (
        "This node provides information about the world's richest billionaires"
        " in 2022"
    ),
    (
        "This node provides information about the world's richest billionaires"
        " in 2021"
    ),
    (
        "This node provides information about the world's richest billionaires"
        " in 2020"
    ),
    (
        "This node provides information on the number of billionaires and"
        " their combined net worth from 2000 to 2023."
    ),
]

#생성된 요약문 별 노드단위 생성
df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

#요약노드 <-> 쿼리엔진 매핑
df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [ ]:
#생성된 노드 확인
df_nodes[0]

In [ ]:
#상위레벨 벡터스토어인덱스 정의
vector_index = VectorStoreIndex(df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

In [ ]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

In [ ]:
response = query_engine.query(
    "What's the net worth of the second richest billionaire in 2023?"
)

In [ ]:
response.source_nodes[0].node.get_content()

In [ ]:
str(response)

In [ ]:
response = query_engine.query("How many billionaires were there in 2009?")

In [ ]:
str(response)

- 서머리 텍스트로 recursive retriever 모듈로 하여금 우리가 찾고자 하는 문서를 자동으로 판별해서 해당 쿼리엔진을 기반으로만 답하게 하는 것이 가능한 것 확인

- 파인콘 DB에 있는 데이터를 연계시켜서도 가능할까?

In [ ]:
!pip install llama-index-vector-stores-pinecone pinecone-client datasets

In [ ]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, PodSpec
import re
import os
os.environ['PINECONE_API_KEY']='<Your API KEY>'
os.environ['OPENAI_API_KEY']='<YOUR API KEY>'
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
from datasets import load_dataset
# 데이터 로드
dataset = load_dataset("lcw99/wikipedia-korean-20221001", split='train[:1000]')
data = dataset.to_pandas()[['id', 'text', 'title']].drop_duplicates(subset='text', keep='first')

In [ ]:
def clean_up_text(content: str) -> str:
    content = re.sub(r'(\w+)-\n(\w+)', r'\1\2', content)

    content = re.sub(r'\\n|  —|——————————|—————————|—————|\\u[\dA-Fa-f]{4}|\uf075|\uf0b7', "", content)

    content = re.sub(r'(\w)\s*-\s*(\w)', r'\1-\2', content)
    content = re.sub(r'\s+', ' ', content)

    return content
from llama_index.core import Document, VectorStoreIndex

documents = [Document(
    text=clean_up_text(row['text']),
    doc_id=row['id'],
    extra_info={'title': row['title']}
) for _, row in data.iterrows()]

In [ ]:
data.head()

In [ ]:
from time import sleep
pc = Pinecone(api_key=pinecone_api_key)
index_name = 'quickstart'
index = pc.Index(index_name)
vector_store = PineconeVectorStore(pinecone_index=index)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
index_name = 'quickstart'
dimension = 1536
metric = 'dotproduct'
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)

while not pc.describe_index(index_name).status['ready']:
    sleep(1)
index = pc.Index(index_name)
sleep(1)
index_stats = index.describe_index_stats()


In [ ]:

from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding
vector_store = PineconeVectorStore(pinecone_index=index)
# 벡터스토어 인덱스에 들어가는 스토리지 컴포넌트 정의
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)
embed_model = OpenAIEmbedding(model='text-embedding-ada-002', embed_batch_size=100)
service_context = ServiceContext.from_defaults(embed_model=embed_model)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,
    service_context=service_context
)

In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
#pc_index = pc.Index(index_name)
#vector_store = PineconeVectorStore(pinecone_index=pc_index)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
agents = {}
for title in data.title:
    vector_query_engine = index.as_query_engine(vector_store_kwargs={"filter": {"title": title}})
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    f"{title}에 대해서 물어볼 때 사용"
                ),
            ),
        ),
    ]

    function_llm = OpenAI(model="gpt-4-turbo-preview", temperature=0)
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
    )

    agents[title] = agent

In [ ]:
#생성된 에이전트 확인
agents

In [ ]:
# 에이전트 선택을 위한 에이전트 서머리
nodes = []
for title in data.title:
    doc_summary = (
        f"이것은 {title}과 관련된 내용이 있습니다. "
        f"{title}과 관련된 내용을 확인하는 용도로 이 인덱스를 사용하세요."
    )
    node = IndexNode(text=doc_summary, index_id=title)
    nodes.append(node)

In [ ]:
# 에이전트 선택 인덱스(노드) 정의
vector_index = VectorStoreIndex(nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

# 에이전트 자체를 쿼리엔진으로 하여 선택된 에이전트가 쿼리 엔진 역할을 하도록 구성
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=agents,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")


query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever,
    response_synthesizer=response_synthesizer
)

In [ ]:
# 해당 문서에서만 답변 가능한 굉장히 구체적인 질문 테스트
response = query_engine.query("셀빅에 대해 알려줘")